<a href="https://colab.research.google.com/github/julieisok/OurFisrtML/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%9A%8D%EB%93%9D_level4_crawling_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 크롤링 1

- 유튜브를 크롤링하여 데이터를 획득
- 크롤링을 해야 이유는 유튜브 자체가 ajax를 이용한 방식이므로 사람의 손길이 필요한
  사이트
- 일반적인 html 자체에는 자체 정의 태그를 들을 혼용한다 => Angular.js를 부분 사용을 했다라고 판단됨.
- 일반적인 css selector의 규칙중에 id규칙(문서내에 고유하다)을 무시한다
  => 사용 주의
- 데이터 로드는 ajax를 통해서 사용자가 스크롤링을 해서 리스트의 끝에 도착할때 더보기(이런 기능)이 처리되므로, 전체 데이터르 수집하기 위해서는 js를 가동시켜야 한다
- 사용라이브러리는 selenium을 사용
- https://www.seleniumhq.org/
- 웹드라이버(여기서는 크롬을 사용)
  > https://sites.google.com/a/chromium.org/chromedriver/downloads  
  > 최신버전이 적용 않되는 혹은 버그가 있는 기능이 잇으므오, 혹시 않되면, 계속 버전을 내려가몃서 사용  
- API-DOCs
  > https://seleniumhq.github.io/selenium/docs/api/py/index.html 
  > https://selenium-python.readthedocs.io/
- install
  > conda install selenium  
  > or  
  > pip| install selenium

In [0]:
from selenium import webdriver as wd

In [0]:
import urllib

In [0]:
# 업로드순 정렬, 오늘(현재기준 24시간이내 게시물)
target_url = 'https://www.youtube.com/results?search_query={}&sp=CAISBAgCEAE%253D'

In [0]:
keyword = '갤럭시노트10'

In [0]:
target_url = target_url.format( urllib.parse.quote(keyword) )
target_url

'https://www.youtube.com/results?search_query=%EA%B0%A4%EB%9F%AD%EC%8B%9C%EB%85%B8%ED%8A%B810&sp=CAISBAgCEAE%253D'

In [0]:
# 웹드라이버 가동
driver = wd.Chrome('../tool/chromedriver_75.0.3770.140.exe')

In [0]:
# 사이트 접속
driver.get( target_url )

In [0]:
# 스크롤를 쳐서 데이터를 다 로드 시킨다
# 데이터량이 엄청 많지 않을때 가능
# ajax를 사용할 경우 서버가 반응해서 응답이 오고
# 화면구성할때까지 시간이 걸린다 스크롤마다 대기시간필요
import time
for n in range(100):
    driver.execute_script('window.scrollBy(0,1000)')
    # 다음 작업을 코드적으로 강제 대기 시킨것
    # 데이터 추출시에는 selenium에서 지원하는 waits 기법을 사용하면 좋다
    time.sleep(2)

In [0]:
# 영상 데이터 한줄 : 
# ytd-video-renderer
# 영상 메타 정보 :
# #video-title

In [0]:
videos = driver.find_elements_by_tag_name('ytd-video-renderer')
len( videos ), type(videos)

(105, list)

In [0]:
#driver.find_element_by_css_selector
for video in videos[:2]:
    # 영상의 타이틀
    print( video.find_element_by_id('video-title').get_attribute('title') )
    # 썸네임
    print( video.find_element_by_tag_name('img').get_attribute('src') )
    # 링크
    print( video.find_element_by_id('video-title').get_attribute('href') )
    # 작성자
    print( video.find_element_by_css_selector('#byline>a').text )
    # 조회수 : metadata-line, # 작성시간
    for span in video.find_elements_by_css_selector('#metadata-line>span'):
        print( span.text)
    '''
    [
      { 'title':'xx', 'thum':'xx', 'link':'xx', 'author':'xx', 'views':x, 'time':''}
      ,{},{},...
    ]
    '''
    

삼성 갤럭시 노트10 실물 색상 비교: 아우라 핑크 vs 아우라 글로우
https://i.ytimg.com/vi/ckGkc7Ypmq0/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLATx5WT8-STFPaguGaTjd_6DAeCKg
https://www.youtube.com/watch?v=ckGkc7Ypmq0
J.H TV
조회수 92회
1시간 전
갤럭시 노트10+ 언박싱 vs 노트10 | 디자인, S펜, 거의 공짜 가격.. 달라진 점 무엇? [구독자 이벤트]
https://i.ytimg.com/vi/_6kv1LR0Rvw/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLATO-xLXqGqmofv4YSm6kTTxcVGRg
https://www.youtube.com/watch?v=_6kv1LR0Rvw
LiveREX라이브렉스
조회수 1.4천회
2시간 전


In [0]:
results = []
for video in videos:#[:2]:    
    dic = {
        'title':video.find_element_by_id('video-title').get_attribute('title')
        ,'thum':video.find_element_by_tag_name('img').get_attribute('src')
        ,'link':video.find_element_by_id('video-title').get_attribute('href')
        ,'author':video.find_element_by_css_selector('#byline>a').text
        ,'views':video.find_elements_by_css_selector('#metadata-line>span')[0].text
        ,'time':video.find_elements_by_css_selector('#metadata-line>span')[1].text
    }
    results.append( dic )
results

[{'title': '삼성 갤럭시 노트10 실물 색상 비교: 아우라 핑크 vs 아우라 글로우',
  'thum': 'https://i.ytimg.com/vi/ckGkc7Ypmq0/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLATx5WT8-STFPaguGaTjd_6DAeCKg',
  'link': 'https://www.youtube.com/watch?v=ckGkc7Ypmq0',
  'author': 'J.H TV',
  'views': '조회수 92회',
  'time': '1시간 전'},
 {'title': '갤럭시 노트10+ 언박싱 vs 노트10 | 디자인, S펜, 거의 공짜 가격.. 달라진 점 무엇? [구독자 이벤트]',
  'thum': 'https://i.ytimg.com/vi/_6kv1LR0Rvw/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLATO-xLXqGqmofv4YSm6kTTxcVGRg',
  'link': 'https://www.youtube.com/watch?v=_6kv1LR0Rvw',
  'author': 'LiveREX라이브렉스',
  'views': '조회수 1.4천회',
  'time': '2시간 전'},
 {'title': "[KT] 드디어 베일을 벗다 '강다니엘의 RED'_full ver_갤럭시 노트 10 5G 아우라레드 / 강다니엘광고",
  'thum': 'https://i.ytimg.com/vi/3Rr7BxQI_NQ/hqdefault.jpg?sqp=-oaymwEZCPYBEIoBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB6n2y_WfCRClRyDsbHDZVnyfBkRQ',
  'link': 'https://www.youtube.com/watch?v=3Rr7BxQI_NQ',
  'author': 'KT - 케이티',
  'views': 

In [0]:
# 여기는 않한다

In [0]:
for link in results[-2:-1]:
    print( link['link'] )
    # 해당 콘텐츠에 진입
    driver.get( link['link'] )
    # 잠시대기 -> 반드시 신규페이지에서는 대기
    print('5초 대기시도')
    driver.implicitly_wait(5)     
    # 화면 스크롤 
    for n in range(3):
        print('화면스크롤 시도')
        driver.execute_script('window.scrollBy(0,1000)')
    print('5초 대기시도')
    time.sleep(5)
    # 후기 개수를 확인 -> 적정선까지 로드
    for cnt in driver.find_elements_by_id('count'):
        print( cnt.text )
    # 후기 채집 (작성자, 내용, 좋아요, 시간)
    print( '-'*20 )
    for comment in driver.find_elements_by_tag_name('ytd-comment-thread-renderer'):        
        print( comment.text )
        print( '-'*20 )
    

https://www.youtube.com/watch?v=P08Z0sDb1TE
5초 대기시도
화면스크롤 시도
화면스크롤 시도
화면스크롤 시도
5초 대기시도

조회수 6,129회
댓글 6개
--------------------
뱃보이
1일 전
우왕 처음으로 댓글1빠당 ><
4
답글
--------------------
매무시
1일 전
우왕 2빠당
2
답글
--------------------
lau 8eom
16시간 전
갖고..싶다... 탈아이폰을 해야하는것인가
1
답글
--------------------
박준형
1일 전
3빠
3
답글
--------------------
손교원
23시간 전
s펜 필기감은 크게 달라진 건 없나요?
답글
--------------------
철팡
23시간 전
초딩들 빠놀이 잼미써?
답글
--------------------


In [0]:
driver.close()
driver.quit()